In [1]:
%load_ext autoreload
%autoreload 2
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Setting OMP and MKL num threads to 1.


Loading BokehJS ...

In [2]:
result_id = 'tless-custom-n_views=1--8865983647'
# result_id = 'tless-siso-n_views=1--684390594'
# ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
ds_name, urdf_ds_name = 'tless.seen.dataset', 'tless.cad'
pred_key = 'pix2pose_detections/refiner/iteration=4'

results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

0:00:01.878613 - Building index and loading annotations...


[Memory]2.3s, 0.0min    : Loading build_index...


In [4]:
results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first()

,view_id
scene_id,
1,153
2,284
3,259
4,447
5,213
6,389
7,306
8,3
9,452


In [5]:
scene_id, view_id = 18, 1

# Replace this here, you can use the dataframe above to get examples of scene/view ids.
# scene_id, view_id = 48, 733

this_preds = filter_predictions(results[pred_key], scene_id, view_id)
# this_preds.poses[0][0][-1] = -this_preds.poses[0][0][-1]
# this_preds.poses[0][1][-1] = -this_preds.poses[0][1][-1]
# this_preds.poses[1][0][-1] = -this_preds.poses[1][0][-1]
# this_preds.poses[1][1][-1] = -this_preds.poses[1][1][-1]
# this_preds.poses[2][0][-1] = -this_preds.poses[2][0][-1]
# this_preds.poses[2][1][-1] = -this_preds.poses[2][1][-1]
# this_preds.poses[3][0][-1] = -this_preds.poses[3][0][-1]
# this_preds.poses[3][1][-1] = -this_preds.poses[3][1][-1]

# opencv_to_pytorch3d = torch.tensor([[-1, 0, 0], [0, -1, 0], [0, 0, 1]], dtype=torch.float, device='cpu')
# for idx in range(len(this_preds)):
#     R = this_preds.poses[idx][:3, :3]
#     T = this_preds.poses[idx][:3, -1]

#     R = R.T
#     R = torch.matmul(R, opencv_to_pytorch3d)
#     T[0] = -T[0]
#     T[1] = -T[1]

#     # this_preds.poses[idx][:3, :3] = R
#     this_preds.poses[idx][:3, -1] = T

# obj_id = 185
# this_preds.poses = torch.unsqueeze(this_preds.poses[obj_id], 0)
# this_preds.poses_input = torch.unsqueeze(this_preds.poses_input[obj_id], 0)
# this_preds.K_crop = torch.unsqueeze(this_preds.K_crop[obj_id], 0)
# this_preds.boxes_rend = torch.unsqueeze(this_preds.boxes_rend[obj_id], 0)
# this_preds.boxes_crop = torch.unsqueeze(this_preds.boxes_crop[obj_id], 0)
# this_preds.infos = this_preds.infos[this_preds.infos.index == obj_id]

renderer = BulletSceneRenderer(urdf_ds_name)
figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
renderer.disconnect()
print(this_preds)

show(figures['input_im'])
show(figures['pred_overlay'])

PandasTensorCollection(
    poses: torch.Size([4, 4, 4]) torch.float32 cpu,
    poses_input: torch.Size([4, 4, 4]) torch.float32 cpu,
    K_crop: torch.Size([4, 3, 3]) torch.float32 cpu,
    boxes_rend: torch.Size([4, 4]) torch.float32 cpu,
    boxes_crop: torch.Size([4, 4]) torch.float32 cpu,
    initial_bboxes: torch.Size([4, 4]) torch.float32 cpu,
----------------------------------------
    infos:
   scene_id  view_id     score       label  det_id  batch_im_id  group_id  \
0        18        1  1.000000  obj_000009   35687            0      8140   
1        18        1  0.802712  obj_000004   35688            0      8140   
2        18        1  0.572549  obj_000001   35689            0      8140   
3        18        1  0.518235  obj_000007   35690            0      8140   

   error  batch_pred_id  
0    NaN          10487  
1    NaN          10488  
2    NaN          10489  
3    NaN          10490  
)
